In [39]:
import pandas as pd 
import numpy as np

## I. Creating a dataFrame with Canada information 
### /!\ You will see the second part below which consists in creating a dataframe with geospatial data 

In [40]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
CodePostalCanada_Html = pd.read_html(link, header = 0) #Header =0 permet de définir la première ligne du tableau comme header
df_CodePostalCanada = CodePostalCanada_Html[0]

df_CodePostalCanada.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [41]:
df_CodePostalCanada.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


##### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [42]:
#Ignore cells with a borough that is Not assigned
df_CodePostalCanadaB = df_CodePostalCanada[df_CodePostalCanada.Borough != 'Not assigned']

In [43]:
df_CodePostalCanadaBGrouped = df_CodePostalCanadaB.groupby(['Postcode'], as_index=False).agg(lambda x :', '.join(x)) 
df_CodePostalCanadaBGrouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough, Scarborough","Rouge, Malvern"
1,M1C,"Scarborough, Scarborough, Scarborough","Highland Creek, Rouge Hill, Port Union"
2,M1E,"Scarborough, Scarborough, Scarborough","Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the code below, I have removed duplicates words (due to join function) in the column named Borough

In [44]:
for row in df_CodePostalCanadaBGrouped.iterrows():
    row[1][1] = row[1][1].split(",")[0]
    # print (row[1][1])

In [45]:
df_CodePostalCanadaBGrouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [46]:
for row in df_CodePostalCanadaBGrouped.iterrows():
    if row[1][2]=='Not assigned' :
        row[1][2] = row[1][1]

In [47]:
df_CodePostalCanadaBGrouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [48]:
df_CodePostalCanadaBGrouped.shape

(103, 3)

## II. Creating a dataFrame with geospatial data

In [49]:
df_Location = pd.read_csv("https://cocl.us/Geospatial_data")

In [50]:
df_Location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Rename the first column to have the same key on both two dataframes: df_CodePostalCanadaBGrouped and df_Location

In [51]:
df_Location.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_Location.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Join the two Dataframe (df_CodePostalCanadaBGrouped and df_Location) on key = Postal Code

In [52]:
df_CodePostalCanada_Location = pd.merge(df_CodePostalCanadaBGrouped, df_Location, on='Postcode')
df_CodePostalCanada_Location.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
